## Calibration

Default Template is given below. The file "config.txt" will be overwritten by calibration script

In [ ]:
%%writefile config.txt
{"template_file": "template.jpg",
          "cap_dev": 0, 
          "top_thresh": 0.75,
          "bottom_thresh": 0.65,
          "fontsize": 0.5,
          "fontweight": 1,
          "box_w": 40,
          "box_h": 40,
          "box_t": 1,
          "box_increments": 20,
          "box_x": 0,
          "box_y": 0,
          "roi_x": 0,
          "roi_y": 0,
          "roi_w": 100,
          "roi_h": 210,
          "msg_x": 10,
          "msg_y": 150,
          "msg_gap": 20,
          "cable_stat": "Ok"
         }


In [ ]:
#This script will create a template from the healthy cable
#TODO: Create ROI and save the settings to a file

import numpy as np
import cv2
import json
import sys
from datetime import datetime


def loadConfig():
    """Load config from file
    The configurations are stored in config.txt in json format [https://www.json.org/]
    The function returns a dictionary similar to first cell"""
    
    config_file = "config.txt"
    try: 
        with open(config_file, 'r') as fp: 
            config_dict = json.load(fp) 
        return config_dict 
    except FileNotFoundError: 
        print("FileNotFoundError: Config.txt")
        sys.exit(1) 

    
def drawText(frame, text, loc):
    """This function will draw a text on the frame at the desired location"""
    cv2.putText(frame, text, loc, font, fontsize, fontcolor, fontweight, fontline)

def saveConfig():
    """Update values and saves dictionary to config file"""
    config["roi_x"] = roi_x
    config["roi_y"] = roi_y
    config["roi_w"] = roi_w
    config["roi_h"] = roi_h
    with open(config_file, 'w') as f:
        json.dump(config, f)
    
    

#update config dict to globals symbol table
globals().update(loadConfig())


font = cv2.FONT_HERSHEY_SIMPLEX
fontline = cv2.LINE_AA
fontcolor = (0, 255, 255)
box_color = (0, 255, 0)
box_x = int(cv2.CAP_PROP_FRAME_WIDTH/2)
box_y = int(cv2.CAP_PROP_FRAME_HEIGHT/2)

message = ""
help_message = ["Arrow Keys: move box", "W,A,S,D: resize box", "r: save ROI file", 
                "space: save template file", "q: quit", "h: toggle help"
                ]
cap = cv2.VideoCapture(cap_dev)
show_help = True


while(cap.isOpened()):
    ret, frame = cap.read()

    if show_help:
        for i, m in enumerate(help_message):
            drawText(frame, m, (5, 25*(i+2)))
            
    drawText(frame, message, (5, 25))
    cv2.rectangle(frame, (box_x, box_y), (box_x + box_w, box_y + box_h), box_color, box_t)
    cv2.imshow("output", frame)

    k = cv2.waitKey(1)
    if k & 0XFF == ord('q'):    # Esc key to stop
        break

    elif k & 0xFF == 81 and box_x != 0:
        #Left Arrow -move the box left
        box_x -= box_increments

    elif k & 0xFF == 82 and box_y != 0:
        #Up Arrow - move the box up
        box_y -= box_increments

    elif k & 0XFF == 83:
        #Right Arrow - move box to right
        box_x += box_increments

    elif k & 0XFF == 84:
        #Down Arrow - move box down
        box_y += box_increments
        
    elif k & 0xFF == ord('a') and box_w != 0:
        #reduce box width
        box_w -= box_increments
        
    elif k & 0xFF == ord('w') and box_h != 0:
        #reduce box height
        box_h -= box_increments
    
    elif k & 0xFF == ord('d'):
        #increase box width
        box_w += box_increments
    
    elif k & 0xFF == ord('s'):
        #increase box height
        box_h += box_increments
    
    elif k & 0xFF == 32:
        #capture template
        template_frame = frame[(box_y+box_t):(box_y+box_h-box_t), (box_x+box_t):(box_x+box_w-box_t)]
        cv2.imshow(template_file, template_frame)
        cv2.imwrite(template_file, template_frame)
        message = "Template file saved!"
   
    elif k & 0xFF == ord('r'):
        #set ROI
        roi_x = box_x + box_t
        roi_y = box_y + box_t
        roi_w = box_w - box_t
        roi_h = box_h - box_t
        roi_frame = frame[roi_y:roi_y+roi_h, roi_x:roi_x+roi_w]
        cv2.imshow("ROI", roi_frame)
        saveConfig()
        message = "ROI saved to config!"
    
    elif k & 0xFF == ord('h'):
        #Toggle Help
        show_help ^= True
        
    else:
        #print(k)
        continue

        

cap.release()
cv2.destroyAllWindows()



#solution for windows not closing bug
for i in range(8):
    cv2.waitKey(1)

## Testing with Healthy rope template

In [ ]:
import serial
import sys
import serial.tools.list_ports as ports

level = "Level : ??"
human_presence = False
lift_stuck = False
fontsize = .75

#Get list of devices and look for devices with name "/dev/ttyACMx"
devices = [comport.device for comport in ports.comports() if comport.device.rfind("ACM") is not -1]
cv2.namedWindow("output", cv2.WINDOW_NORMAL)
cv2.namedWindow("xcorr", cv2.WINDOW_NORMAL)
cv2.resizeWindow("output", 500,400)
cv2.resizeWindow("xcorr", 300,300)

for p in devices:
    try:
        ser = serial.Serial(p, 9600, timeout=0)
        connected = True
        break
    except SerialException:
        print("SerialException: ")
        print("Couldn't connect to serial port")

template = cv2.imread(template_file ,0)

cap = cv2.VideoCapture(cap_dev)
box_h, box_w = template.shape

fontcolor = (0, 255, 0)


while(cap.isOpened()):
    if ser.is_open:
        lvl = ser.readline().decode('utf-8')
        if lvl.rfind("Level :") != -1:
            level = lvl
        if lvl.rfind("DANGER") != -1:
            human_presence = True
        if lvl.rfind("WARNING") != -1:
            lift_stuck = True
 
    ret, bgr_frame = cap.read()
    frame = cv2.cvtColor(bgr_frame, cv2.COLOR_BGR2GRAY)
    res = cv2.matchTemplate(frame, template, cv2.TM_CCOEFF_NORMED)
    cv2.imshow('xcorr', res)
 

    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
    if max_val >= top_thresh:
        top_left = max_loc
        bottom_right = (top_left[0] + box_w, top_left[1] + box_h)
        cable_stat = "Cable ok!"
        fontcolor = (0, 255, 0)
        cv2.rectangle(bgr_frame, top_left, bottom_right, box_color, 2)
        
    if max_val <= bottom_thresh:
        #No good match
        cable_stat = "Bad Cable!"
        fontcolor = (0, 0, 255)
        
        
    
    date = datetime.now().strftime('%m-%d %H:%M:%S')
    message = [ "max: {:.4f}".format(max_val), "time: {}".format(date), 
               "stat: {}".format(cable_stat), "{}".format(level)
              ]
    for i, msg in enumerate(message):
        drawText(bgr_frame, msg ,(5, 25*(i+1)))
    
    if (human_presence):
        fontcolor = (0, 0, 255)
        drawText(bgr_frame, "DANGER: Human presence!", (200, 200))
    
    if (lift_stuck):
        fontcolor = (0, 255, 255)
        drawText(bgr_frame, "WARNING: Lift Stuck!!", (200, 300))
  
    cv2.imshow("output", bgr_frame)
    k = cv2.waitKey(1)
    if k & 0xFF == ord('q'):
        break

    elif k & 0XFF == 49:
        #1 - b2 -move box down
        ser.write(b'b1;')
        lift_stuck = False
        human_presence = False
        
    elif k & 0xFF == 50:
        #2 - move the box up
        ser.write(b'b2;')
        lift_stuck = False
        human_presence = False
    
    elif k & 0XFF == 51:
        #3 -b3 - move box down
        ser.write(b'b3;')
        lift_stuck = False
        human_presence = False
 
    elif k & 0xFF == ord('w'):
        #move up for 1 sec
        ser.write(b'++;')
    
    elif k & 0xFF == ord('s'):
        #move down for 1 sec
        ser.write(b'--;')
    
    else:
        continue
        

cap.release()
cv2.destroyAllWindows()


#solution for windows not closing bug
for i in range(8):
    cv2.waitKey(1)
    
#close serial
ser.close()